# Working with Alignments in CITE 2

## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [ ]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [ ]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`
import $ivy.`edu.holycross.shot::greek:2.3.3`


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [ ]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.greek._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source


## Useful Functions

Save a string:

In [ ]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

Pretty Print many things:

In [ ]:
def showMe(v:Any):Unit = {
  v match {
    case _:StringHistogram => {
        for ( h <- v.asInstanceOf[StringHistogram].histogram ) {
            println(s"${h.count}\t${h.s}")
        }
    }
  	case _:Corpus => {
  		for ( n <- v.asInstanceOf[Corpus].nodes) {
  			println(s"${n.urn.passageComponent}\t\t${n.text}")
  		}	
  	}
    case _:Vector[Any] => {
       for (a <- v.asInstanceOf[Vector[Any]]){
           a match {
               case _:Corpus => {
                println("---------")
                  println( s"${a.asInstanceOf[Corpus].nodes.head.urn.dropPassage}")
                  println("")
                  showMe(a)
                  println("")
               }
               case _:CtsUrn => println(s"${a}")
               case _:Cite2Urn => println(s"${a}")
               case _ => {
                   println("-----------")
                   showMe(a)
               }
           }
       }
    }
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

## Load Library

We will load a Version-level, bilingual file, to start:

In [ ]:
val cexPath = "cex/Catullus1-aligned.cex"
val lib = CiteLibrary(scala.io.Source.fromFile(cexPath).mkString)

Get parts of the library where we can use them:

In [ ]:
lazy val tr: TextRepository = lib.textRepository.get
lazy val corp: Corpus = tr.corpus
lazy val cat: Catalog = tr.catalog

## Alignment-Specific Setup

Alignment collections will be recorded as belonging to this data-model:

In [ ]:
val alignModel = Cite2Urn("urn:cite2:cite:datamodels.v1:alignment")

This is the CITE Relations verb that “glues” passages to a given alignment:

In [ ]:
val alignVerb = Cite2Urn("urn:cite2:cite:verbs.v1:aligns")

We can get a Vector of Alignment-collections:

In [ ]:
val alignmentCollections: Vector[Cite2Urn] = lib.collectionsForModel(alignModel)

## Aligmnment-Specific Functions

(These functions assume values defined above. They do not do elaborate checking for necessary components of a CITE Library, for example. They are not ready to be abstracted out of this notebook!)

### `alignmentsForPassage( psg: CtsUrn ): Vector[Cite2Urn]`

In [ ]:
def alignmentsForPassage( psg: CtsUrn ): Vector[Cite2Urn] = {
    
    val crs: CiteRelationSet = lib.relationSet.get.verb(alignVerb) // assumption the Option is not None!
    // The "psg" might be a range or a container, so expand it
    val allPsgs: Vector[CtsUrn] = corp.validReff(psg) :+ psg distinct
    
    val allRelations: Set[CiteTriple] = crs.relations.filter( t => {
        allPsgs.contains( t.urn2.asInstanceOf[CtsUrn])
    })
    
    allRelations.map(_.urn1.asInstanceOf[Cite2Urn]).toVector.distinct
}

Test:

In [ ]:
val testPsgU = CtsUrn("urn:cts:latinLit:phi0472.phi001.merrill.token:1.4.1")
val testAlignments: Vector[Cite2Urn] = alignmentsForPassage( testPsgU )
assert ( testAlignments.size == 2 )
assert ( testAlignments.contains( Cite2Urn("urn:cite2:ducat:alignments.temp:20191012_31_11_675_0") ))

### `passagesForAlignment(  alignment: Cite2Urn  ): Vector[CtsUrn]`

In [ ]:
def passagesForAlignment(  alignment: Cite2Urn  ): Vector[CtsUrn] = {
    val crs: CiteRelationSet = lib.relationSet.get.verb(alignVerb) // assumption the Option is not None!
    val rels: CiteRelationSet = crs.urn1Match(alignment)
    val unsortedUrns: Vector[CtsUrn] = rels.relations.map( r => {
        r.urn2.asInstanceOf[CtsUrn]
    }).toVector
    corp.sortPassages(unsortedUrns)
}

def passagesForAlignment( alignments: Vector[Cite2Urn] ): Vector[CtsUrn] = {
    val psgs: Vector[CtsUrn] = alignments.map( a => {
        passagesForAlignment( a )
    }).flatten
    corp.sortPassages(psgs)
}

Test:

In [ ]:
val testPassages: Vector[CtsUrn] = passagesForAlignment( testAlignments.head )
assert( testPassages.size == 2 )


### `textsForAlignment( alignment: Cite2Urn): Vector[Corpus]`

In [ ]:
def textsForAlignment( alignment: Cite2Urn): Vector[Corpus] = {
    val psgs: Vector[CtsUrn] = passagesForAlignment( alignment )
    (corp ~~ psgs).chunkByText
}

def textsForAlignment( alignments: Vector[Cite2Urn] ): Vector[Corpus] = {
    val psgs: Vector[CtsUrn] = alignments.map( a => {
        passagesForAlignment(a)
    }).flatten
    (corp ~~ psgs).chunkByText
}

Test:

In [ ]:
val testTexts: Vector[Corpus] = textsForAlignment( testAlignments.head )
assert( testTexts.size == 2 )
showMe(testTexts)

### `alignedTexts( psg: CtsUrn ): Vector[Corpus]`

In [ ]:
def alignedTexts( psg: CtsUrn ): Vector[Corpus] = {
    val alignments: Vector[Cite2Urn] = alignmentsForPassage(psg)
    val texts: Vector[Corpus] = textsForAlignment( alignments )
    texts
}

def alignedTexts( psgs: Vector[CtsUrn] ): Vector[Corpus] = {
    
    val alignments: Vector[Cite2Urn] = psgs.map ( p => {
        alignmentsForPassage(p)
    }).flatten
    val texts: Vector[Corpus] = textsForAlignment( alignments )
    texts
}

Test:

In [ ]:
val testAlignedTexts1: Vector[Corpus] = alignedTexts(testPsgU)
assert( testAlignedTexts1.size == 2 )
showMe(testAlignedTexts1)

### `alignmentsForString( s: String, tokenMatch: Boolean = true ): Vector[Corpus]`

In [ ]:
def alignmentsForString( s: String, tokenMatch: Boolean = true ): Vector[Cite2Urn] = {
    val strCorpus: Corpus = {
        if (tokenMatch) {
            val nodes: Vector[CitableNode] = corp.nodes.filter( _.text.contains(s))
            Corpus(nodes)
        } else {
            corp.find(s)
        }
    }
    
    val strUrns: Vector[CtsUrn] = strCorpus.urns
    
    val crs: CiteRelationSet = {
      val allCrs =  lib.relationSet.get.verb(alignVerb) // assumption the Option is not None!
    
      val foundSet = allCrs.relations.filter( u2 => {
          strCorpus.urns.contains(u2.urn2.asInstanceOf[CtsUrn])
      }).toSet
      CiteRelationSet(foundSet)
    }
    val foundAlignments: Vector[Cite2Urn] = crs.relations.map( a => {
        a.urn1.asInstanceOf[Cite2Urn]
    }).toVector.distinct  
    foundAlignments
}

Test:

In [ ]:
val testString = "you"
val testStringAlignments: Vector[Cite2Urn] = alignmentsForString(testString)
showMe(testStringAlignments)

# Playground

## Catullus-Specific Things

Make URN work easier:

In [ ]:
val catullusUrn = CtsUrn("urn:cts:latinLit:phi0472.phi001:")
val latUrn = CtsUrn("urn:cts:latinLit:phi0472.phi001.merrill:")
val engUrn = CtsUrn("urn:cts:latinLit:phi0472.phi001.ozlam:")


## See Alignments for a Passage

**Edit This!** A passage of Catullus 1:

In [ ]:
val myPassage = "1.1"

**See Alignments!**

In [ ]:
val myAlignedTexts: Vector[Corpus] = alignedTexts( catullusUrn.addPassage(myPassage))

showMe(myAlignedTexts)

**Dynamic Lexicon!** Edit your preferred word:

In [ ]:
val myWord = "you"
val dynLex: Vector[Cite2Urn] = alignmentsForString( myWord )

Show the results:

In [ ]:
for ( dl <- dynLex) {
    println(s"==========\nAlignment\n")
    println(s"${dl}")
    val texts: Vector[Corpus] = textsForAlignment(dl)
    showMe(texts)
}